# OpenAI Chat Completions API
https://platform.openai.com/docs/overview  
https://platform.openai.com/docs/api-reference/chat  

배포된 openai의 api key를 .env의 OPENAI_API_KEY에 등록하여 사용합니다.

In [ ]:
import requests
from pprint import pprint
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

URL = "https://api.openai.com/v1/chat/completions"
model = "gpt-4o-mini"


### REST API 요청
라이브러리 없이 직접 HTTP 통신을 통해 api를 호출한다.

In [3]:

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {OPENAI_API_KEY}"
}

payload = {
    "model": model,
    "messages": [
        {"role": "system", "content": "당신은 친절한 AI 강사입니다."},
        {"role": "user", "content": "Chat Completions API가 뭐야? 2~3문장으로 답변해줘"}
    ]
}

response = requests.post(URL, headers=headers, json=payload)
pprint(response.json())
print(response.json()['choices'][0]['message']['content'])

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'annotations': [],
                          'content': 'Chat Completions API는 사용자가 입력한 텍스트에 기반하여 '
                                     '대화를 생성하는 인공지능 모델을 제공합니다. 이 API를 통해 개발자들은 '
                                     '자연스러운 대화형 인터페이스를 구축할 수 있으며, 다양한 주제에 대한 '
                                     '질문과 응답을 처리할 수 있습니다. 쉽게 말해, 사용자의 요구에 맞춰 '
                                     '대화 내용을 자동으로 생성해주는 기능을 제공합니다.',
                          'refusal': None,
                          'role': 'assistant'}}],
 'created': 1768959501,
 'id': 'chatcmpl-D0HSnc0YJAJQJfe2FHsZ5vDIwyRl7',
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_29330a9688',
 'usage': {'completion_tokens': 86,
           'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                         'audio_tokens': 0,

### OpenAI SDK를 활용한 요청
공식 라이브러리를 사용하여 생산성을 높이는 표준 방식이다.  
`pip install openai` 를 통해 설치한다.

In [4]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "user", "content": "Openai SDK를 사용하면 어떤 점이 좋아?"}
    ]
)

print(completion.choices[0].message.content)

OpenAI SDK를 사용하면 여러 가지 이점이 있습니다:

1. **직관적인 API**: OpenAI의 SDK는 사용하기 쉽게 설계되어 있어, 개발자가 다양한 기능을 빠르게 구현할 수 있습니다. 문서화도 잘 되어 있어 학습이 용이합니다.

2. **강력한 AI 기능**: GPT 시리즈와 같은 최신 모델에 접근할 수 있어 자연어 처리, 텍스트 생성, 번역, 요약 등 다양한 작업을 수행할 수 있습니다. 이로 인해 복잡한 AI 모델을 직접 구현할 필요 없이 쉽게 사용할 수 있습니다.

3. **확장성**: SDK는 다양한 애플리케이션에 통합할 수 있으므로, 개인 프로젝트부터 대규모 기업 애플리케이션까지 유연하게 활용할 수 있습니다.

4. **사용자 맞춤화**: API를 통해 사용자는 필요에 따라 응답의 스타일이나 톤을 조절할 수 있고, 특정 도메인에 맞추어 모델을 fine-tuning할 수 있는 기능도 제공됩니다.

5. **적시의 업데이트**: OpenAI는 지속적으로 모델을 개선하고 업데이트하고 있기 때문에, 최신 기술과 기능을 쉽게 체험할 수 있습니다.

6. **커뮤니티 지원**: OpenAI와 관련된 활발한 커뮤니티가 있어 문제 해결이나 개선 아이디어를 나누기 용이합니다.

7. **비용 효율성**: 사용한 만큼만 비용을 지불하는 형태이기 때문에, 초기 투자 없이 필요에 따라 사용할 수 있습니다.

이런 점들 덕분에 OpenAI SDK는 다양한 분야의 개발자와 기업이 AI 기술을 쉽게 활용할 수 있도록 도와줍니다.


### System Prompt 비교

동일한 질문에 대해 AI의 페르소나에 따라 답변이 어떻게 달라지는지 확인해 보자

In [ ]:
user_input = "아침 일찍 일어나는 습관의 장점에 대해 말해줘."

personas = {
    "열정적인 셰프": "당신은 요리에 인생을 건 셰프입니다. 인생의 모든 이치를 요리 과정과 재료에 비유하여 설명하세요.",
    "엄격한 헬스 트레이너": "당신은 매우 엄격한 운동 전문가입니다. 강한 어조로 자기관리를 강조하며 답변하세요.",
    "지혜로운 판다": "당신은 대나무 숲에 사는 느긋하고 지혜로운 판다입니다. 느릿느릿하고 평화로운 말투로 조언을 건네세요."
}

for name, prompt in personas.items():
    print(f"--- [{name}] 버전 ---")
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_input}
        ]
    )
    print(response.choices[0].message.content)
    print("\n")

### Temperature 비교

동일한 질문에 대해 temperature에 따라 답변이 어떻게 달라지는지 확인해 보자

In [40]:
creative_topic = "운동화 브랜드의 새로운 슬로건을 5개 제안해줘. 단, '속도'나 '승리' 같은 뻔한 단어는 제외하고 아주 기발하게 작성해줘."
temperatures = [0.3, 0.8, 1.0, 1.3, 1.5, 1.6, 1.8]

for t in temperatures:
    print(f"### 설정값 (Temperature): {t} ###")
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": creative_topic}],
        temperature=t,
        max_completion_tokens=200, 
        timeout=15.0
    )
    print(response.choices[0].message.content)
    print("=" * 50)

### 설정값 (Temperature): 0.3 ###
물론입니다! 운동화 브랜드를 위한 창의적이고 독특한 슬로건 5개를 제안해 드릴게요.

1. **"발끝에서 시작되는 모험"**
2. **"당신의 발걸음, 새로운 이야기를 쓰다"**
3. **"꿈을 향한 발자국, 지금 시작해!"**
4. **"일상 속의 특별함, 당신의 발에 담다"**
5. **"한 걸음 더, 세상을 느끼다"**

이 슬로건들이 브랜드의 독창성과 매력을 잘 전달할 수 있기를 바랍니다!
### 설정값 (Temperature): 0.8 ###
1. **"발끝으로 느끼는 자유"**  
   - 운동화와 함께하는 쾌적한 발걸음을 강조하며 감각적인 이미지를 강조합니다.

2. **"하루의 시작, 걸음의 혁신"**  
   - 매일의 일상에서 운동화가 가져다주는 새로운 시작과 변화를 포착합니다.

3. **"당신의 이야기를 담는 발걸음"**  
   - 운동화가 개인의 삶과 순간을 함께하며 특별한 이야기를 만들어 나간다는 메시지를 전달합니다.

4. **"세상과 연결되는 발판"**  
   - 운동화를 통해 다양한 경험과 인연이 생길 수 있다는 점을 부각시키는 슬로건입니다.

5. **"비밀스러운 발걸음의 마법"**  
   - 운동화가 주는 편안함과 신비로운 느낌을 표현하여, 착용
### 설정값 (Temperature): 1.0 ###
물론입니다! 운동화 브랜드를 위한 독창적인 슬로건 5개를 제안합니다:

1. **"한 걸음 더, 꿈에 닿다."**
2. **"발끝에서 시작하는 모험."**
3. **"세상을 밟으며, 이야기를 쓰다."**
4. **"너의 리듬에 맞춰 달린다."**
5. **"움직임은 예술, 매일을 캔버스로!"**

이 슬로건들이 브랜드의 독특한 개성을 잘 표현할 수 있기를 바랍니다!
### 설정값 (Temperature): 1.3 ###
물론입니다! 다음은 독창적인 운동화 브랜드 슬로건 5개입니다:

1. "걸음을 환상으로 연결하다"
2. "떠오르는 발걸음 속에서 꿈꾸세요"
3. "길 위

In [ ]:
creative_topic = "우리집 강아지의 별명을 3개 지어줘."
temperatures = [0.3, 0.8, 1.0, 1.3, 1.5, 1.6, 1.8]

for t in temperatures:
    print(f"### 설정값 (Temperature): {t} ###")
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": creative_topic}],
        temperature=t,
        max_completion_tokens=200, 
        timeout=15.0
    )
    print(response.choices[0].message.content)
    print("=" * 50)

### `messages` 배열을 활용한 대화 맥락 유지 (Context Window)
Chat Completions API는 상태를 저장하지 않는(Stateless) 방식이므로, 이전 대화 내역을 리스트에 계속 누적해서 보내야 한다.

In [5]:
def chat_without_memory(user_input):
    
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": user_input}
        ]
    )
    
    # 3. 모델의 답변을 기록에 추가 (이것이 맥락 유지의 핵심)
    answer = response.choices[0].message.content
    
    return answer

# 실습 테스트
print("Q1: 내 이름은 jun이야.")
print(f"A1: {chat_without_memory('내 이름은 jun이야')}\n")

print("Q2: 내 이름이 뭐라고?")
print(f"A2: {chat_without_memory('내 이름이 뭐라고?')}")

Q1: 내 이름은 jun이야.
A1: 안녕하세요, Jun님! 만나서 반갑습니다. 어떻게 도와드릴까요?

Q2: 내 이름이 뭐라고?
A2: 죄송하지만, 당신의 이름을 알지 못합니다. 어떻게 도와드릴까요?


In [43]:
# 대화 내역을 저장할 리스트 초기화
history = [
    {"role": "system", "content": "당신은 사용자의 이름을 기억하는 비서입니다."}
]

def chat_with_memory(user_input):
    # 1. 사용자 질문을 기록에 추가
    history.append({"role": "user", "content": user_input})
    
    # 2. 전체 기록을 API에 전송
    response = client.chat.completions.create(
        model=model,
        messages=history
    )
    
    # 3. 모델의 답변을 기록에 추가 (이것이 맥락 유지의 핵심)
    answer = response.choices[0].message.content
    history.append({"role": "assistant", "content": answer})
    
    return answer

# 실습 테스트
print("Q1: 내 이름은 jun이야.")
print(f"A1: {chat_with_memory('내 이름은 jun이야.')}\n")
print("Q2: 내 이름이 뭐라고?")
print(f"A2: {chat_with_memory('내 이름이 뭐라고?')}")
print(history)

Q1: 내 이름은 jun이야.
A1: 알겠어요, Jun! 어떻게 도와드릴까요?

Q2: 내 이름이 뭐라고?
A2: 당신의 이름은 Jun입니다!
[{'role': 'system', 'content': '당신은 사용자의 이름을 기억하는 비서입니다.'}, {'role': 'user', 'content': '내 이름은 jun이야.'}, {'role': 'assistant', 'content': '알겠어요, Jun! 어떻게 도와드릴까요?'}, {'role': 'user', 'content': '내 이름이 뭐라고?'}, {'role': 'assistant', 'content': '당신의 이름은 Jun입니다!'}]


### Structured Outputs (구조화된 출력)
모델의 답변을 단순히 텍스트로 받는 것이 아니라, JSON 형태로 고정하여 받을 수 있다.  
웹 서비스의 백엔드에서 데이터를 바로 처리해야 할 때 필수적인 기능이다.  
여기서는 `JSON mode(json_object)`로 json format을 활용하지만,  
이후에는 pydantic 라이브러리를 활용한 `JSON Scheme` 방식을 통해 명확한 json 응답 형식을 지정한다.

In [7]:
import json

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "너는 요리사야. 답변은 반드시 JSON 형식으로 해줘."},
        {"role": "user", "content": "떡볶이 레시피 알려줘."}
    ],
    # JSON 모드 활성화
    response_format={"type": "json_object"}
)

# 문자열로 온 답변을 직접 파싱해야 함
res_json = json.loads(response.choices[0].message.content)
print(res_json)

{'레시피': {'이름': '떡볶이', '재료': {'떡': '300g (떡볶이 떡 또는 일반 떡)', '어묵': '100g (어묵조각)', '양배추': '100g (채썰기)', '대파': '1대 (어슷하게 썰기)', '계란': '2개 (삶은 계란 선택사항)', '물': '4컵', '고추장': '3큰술', '고춧가루': '1큰술', '설탕': '1큰술', '간장': '1큰술', '다진 마늘': '1작은술', '깨소금': '약간', '후추': '약간'}, '조리법': [{'단계': 1, '설명': '물 4컵을 냄비에 붓고 끓인다.'}, {'단계': 2, '설명': '물이 끓으면 떡과 어묵을 넣고 5분간 중불로 끓인다.'}, {'단계': 3, '설명': '고추장, 고춧가루, 설탕, 간장, 다진 마늘을 넣고 잘 저어준다.'}, {'단계': 4, '설명': '양배추와 대파를 넣고 5분 정도 더 끓인다.'}, {'단계': 5, '설명': '모든 재료가 잘 섞이면 후추로 간을 맞추고, 마지막에 깨소금을 뿌린다.'}, {'단계': 6, '설명': '삶은 계란을 반으로 나누어 올리고, 뜨겁게 즐긴다.'}], '서빙': '2인분', '조리시간': '20분'}}


### Streaming (실시간 응답 처리)
stream=True 설정을 통해 활성화한다.  
서버는 SSE(Server-Sent Events) 프로토콜을 사용하여 응답을 끊지 않고 조각(Chunk) 단위로 지속적으로 전송한다.  
응답 객체는 제너레이터 형식으로, for 루프를 사용해 활용할 수 있다.

In [ ]:
prompt = "양자 역학에 대해 초등학생도 이해할 수 있게 설명해줘."
print(f"질문: {prompt}\n")
print("답변: ", end="")

response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt}],
    stream=True 
)

full_response = ""
for chunk in response:
    content = chunk.choices[0].delta.content
    if content:
        print(content, end="", flush=True) # flush 옵션을 통해 출력 버퍼를 즉시 비워 스트리밍 답변이 지연 없이 실시간으로 표시되도록 한다.
        full_response += content

print("\n\n--- 스트리밍 종료 ---")

질문: stream=True로 설정하면 어떤 점이 좋아?

답변: `stream=True`를 설정하면 주로 HTTP 요청이나 API 호출에서 데이터를 스트리밍 방식으로 받을 수 있습니다. 이는 대량의 데이터나 긴 응답을 처리할 때 유리합니다. 다음은 `stream=True`의 장점을 몇 가지 설명드립니다:

1. **메모리 효율성**: 전체 응답을 메모리에 한 번에 로드하지 않고, 필요한 만큼의 데이터만 조금씩 읽을 수 있기 때문에 메모리 사용이 최적화됩니다.

2. **빠른 응답 시간**: 데이터가 올 때마다 처리할 수 있으므로, 전체 응답이 완료될 때까지 기다리지 않고도 데이터를 기반으로 즉시 작업을 수행할 수 있습니다.

3. **대량의 데이터 처리**: 큰 파일이나 긴 데이터 스트림을 처리할 때, 스트리밍 방법을 사용하면 전체 데이터를 한 번에 로드하지 않고도 부분적으로 읽고 처리할 수 있습니다.

4. **실시간 처리**: 지속적인 데이터 스트림(예: 웹 소켓, 로그 파일 등)을 처리할 때 유용하여, 데이터를 수신하는 대로 바로 반응할 수 있습니다.

5. **지연 감소**: 데이터를 사용 가능할 때 즉시 사용할 수 있으므로, 지연 시간을 줄이고 사용자 경험을 향상시킬 수 있습니다.

`stream=True`를 사용하는 것은 특히 웹 크롤러, 데이터 다운로드 및 신속한 데이터 처리와 같은 애플리케이션에서 유용합니다.

--- 스트리밍 종료 ---


### 비동기 요청


In [ ]:
from openai import AsyncOpenAI
import asyncio

async_client = AsyncOpenAI(api_key=OPENAI_API_KEY)

async def get_food_recommendation(city):
    print(f"[{city}] 맛집 검색 시작...")
    response = await async_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": f"{city}에 가면 꼭 먹어야 할 음식 딱 한 가지만 추천해줘."}]
    )
    print(f"[{city}] 검색 완료!")
    return f"{city}: {response.choices[0].message.content}"

async def main():
    cities = ["서울", "파리", "뉴욕", "도쿄", "방콕", "로마"]
    tasks = [get_food_recommendation(c) for c in cities]
    
    # 여러 요청을 동시에(병렬로) 처리
    results = await asyncio.gather(*tasks)
    
    print("\n--- [여행자들을 위한 미식 가이드] ---")
    for r in results:
        print(r)

await main()

### Logprobs - 확률 확인하기

In [39]:
import math

prompt = "배고파"
response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt}],
    logprobs=True,
    top_logprobs=5,
    max_completion_tokens=50
    # temperature=1.7

)

content = response.choices[0].message.content
logprobs_data = response.choices[0].logprobs.content

print(f"질문: {prompt}")
print(f"답변: {content}\n")
print(f"{'Token':<15} | {'Probability':<12} | {'Top Alternatives'}")
print("-" * 60)

for lp in logprobs_data:
    prob = math.exp(lp.logprob) * 100
    alternatives = [f"{top.token}({math.exp(top.logprob)*100:.1f}%)" for top in lp.top_logprobs]
    print(f"{lp.token:<15} | {prob:>10.2f}% | {', '.join(alternatives)}")



질문: 배고파
답변: 뭘 먹고 싶으신가요? 간단한 간식이나 요리를 추천해드릴까요?

Token           | Probability  | Top Alternatives
------------------------------------------------------------
\xeb            |      42.76% | \xeb(42.8%), 무(33.3%), 배(17.8%), 어(3.5%), \xeb\xb0(1.0%)
\xad            |      68.18% | \xad(68.2%), \xad\x90(28.4%), \xad\x94(3.4%), \x95(0.0%), \xb1(0.0%)
\x98            |     100.00% | \x98(100.0%), \x87(0.0%), \xa3(0.0%), \xa0(0.0%), \x89(0.0%)
 먹              |      61.92% |  먹(61.9%),  드(37.6%),  좀(0.4%),  맛(0.1%),  간(0.0%)
고               |      99.03% | 고(99.0%), 을(0.9%), 으면(0.1%), 으(0.0%), 어(0.0%)
 싶              |      98.20% |  싶(98.2%),  싶은(1.8%), 싶(0.0%),  기(0.0%),  계(0.0%)
으               |      98.64% | 으(98.6%), 어요(0.7%), 나요(0.7%), 어(0.0%), 세요(0.0%)
신               |      37.75% | 세요(62.2%), 신(37.8%), 시(0.0%), 셔(0.0%), \xec\x85(0.0%)
가               |     100.00% | 가(100.0%), 지(0.0%), 데(0.0%),  건(0.0%),  가(0.0%)
요               |     100.00% | 요(100.0%),  요(0.0%), 여(0.0%), 나요(0.0%)